# Deploy the bot in prod mode

Run this notebook on the serverless cluster.

## Pre-deployment agent validation
Before registering and deploying the agent, perform pre-deployment checks using the [mlflow.models.predict()](https://mlflow.org/docs/latest/python_api/mlflow.models.html#mlflow.models.predict) API. See Databricks documentation ([AWS](https://docs.databricks.com/en/machine-learning/model-serving/model-serving-debug.html#validate-inputs) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/machine-learning/model-serving/model-serving-debug#before-model-deployment-validation-checks)).

In [0]:
%run ../../../libs/botops/buildsetup_serverless

In [0]:
import mlflow

In [0]:
TRIPBOT_RUN_ID = "2acb51a1d21b45339d7f479e71063a67"  # Replace run id with the id for the run where you logged the model
BOT_NAME = "tripbot"

In [0]:
mlflow.models.predict(
    model_uri=f"runs:/{TRIPBOT_RUN_ID}/{BOT_NAME}",
    input_data={"messages": [{"role": "user", "content": "Hello!"}]},
    env_manager="uv",
)

## Register the model to Unity Catalog

Before you deploy the agent, you must register the agent to Unity Catalog.

Unity catalog is Databricks universal resource registry, somewhat similar to a Data Warehouse Schema.

We must the `catalog`, `schema`, and `model_name` below to register the MLflow model to Unity Catalog.

In [0]:
# Name functions enables automatic env+user specific database naming
from brickops.datamesh.naming import catname_from_path
from brickops.datamesh.naming import dbname

In [0]:
cat = catname_from_path()
db = dbname(db="taxinycbots", cat=cat, env="prod")
print("New db name: " + db)
spark.sql(f"USE catalog {cat}")
spark.sql(f"CREATE DATABASE IF NOT EXISTS {db}")

#### Register the model

Register the model un Unity catalog

In [0]:
mlflow.set_registry_uri("databricks-uc")

UC_MODEL_NAME = f"{db}.{BOT_NAME}"
model_uri = f"runs:/{TRIPBOT_RUN_ID}/{BOT_NAME}"
# register the model to UC
uc_registered_model_info = mlflow.register_model(model_uri=model_uri, name=UC_MODEL_NAME)

In [0]:
# Show the model info
uc_registered_model_info

In [0]:
# Print current version
uc_registered_model_info.version

### Task: What version of the bot did you deploy?

Each deploy will get a new version.

## Deploy the model

It can take up to 15m for the model to go live.

If a model already exists, you might get 409 or similar errors.

You can try the review app for the production model [here](https://dbc-639f4875-165d.cloud.databricks.com/ml/review/transport.taxinycbots.tripbot/1/).

In [0]:
from databricks import agents
agents.deploy(UC_MODEL_NAME, uc_registered_model_info.version, tags = {"endpointSource": "docs"})

## Next steps

After your agent is deployed, you could chat with it in AI playground to perform additional checks, share it with SMEs in your organization for feedback, or embed it in a production application. See docs ([AWS](https://docs.databricks.com/en/generative-ai/deploy-agent.html) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/deploy-agent)) for details